# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 

In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Load the environment variable `PRICE_DATA`
PRICE_DATA = os.getenv('PRICE_DATA')

# Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Read files
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
# group values by ticker, sort values by date
dd_pz = dd_px.groupby('ticker', group_keys=False).apply(lambda x: x.sort_values("Date"))

# Add lags for variables Close and Adj_Close.
#dd_px.columns # Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source', 'Year'], dtype='object')
dd_shift = dd_pz.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1),
                       Adj_Close_lag_1 = x['Adj Close'].shift(1))
)
# Add returns based on Close:          `returns`: (Close / Close_lag_1) - 1
dd_rets = dd_shift.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(returns = x['Close']/x['Close_lag_1'] - 1)
)
# Add the following range: `hi_lo_range`: this is the day's High minus Low
dd_hi_lo_range = dd_rets.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(hi_lo_range = x['High'] - x['Low'])
)
# Assign the result to `dd_feat`
dd_feat = dd_hi_lo_range

#dd_feat
#dd_feat.compute()
#type(dd_feat.compute())
#len(dd_feat) # 359088
#len(dd_feat.columns) #13


C:\Users\The Winner\AppData\Local\Temp\ipykernel_11120\159928415.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_pz = dd_px.groupby('ticker', group_keys=False).apply(lambda x: x.sort_values("Date"))
C:\Users\The Winner\AppData\Local\Temp\ipykernel_11120\159928415.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_pz.groupby('ticker', group_keys=False).apply(
C:\Users\The Winner\AppData\Local\Temp\ipykernel_11120\159928415.py:13: UserWarning: `meta` is not specified, inferred from partial data. P

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [5]:
# Convert the Dask data frame to a pandas data frame
pd_feat = dd_feat.compute().reset_index()
# Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`
pd_feat= pd_feat.groupby('ticker').apply(
    lambda x: x.assign(mov_avg = x['returns'].rolling(10).mean())
)
# remove extra index created
pd_feat.reset_index(drop=True, inplace=True)
#pd_feat

C:\Users\The Winner\AppData\Local\Temp\ipykernel_11120\601101431.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pd_feat= pd_feat.groupby('ticker').apply(


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

No, we could have done it with Dask
+ Would it have been better to do it in Dask? Why?

In Dask, there is bigger overhead as it does paralelization and prepares the processes, but then it calculates it faster.

It is a trade off between overhead time vs calculation saved time.

This size, likely it would have been better to use Dask. 

See below code to do the moving average feature. In my computer, doing it with Pandas takes 16 seconds, doing it with Dask shows 0.0 seconds if not computed. 

It doesn't surprise me, in the past I was encountering 100x performance improvements with Numpy arrays instead of Pandas dataframes.

(1 pt)

In [186]:
# In Dask, add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`
dd_feat_mov_avg = dd_feat.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(mov_avg = x['returns'].rolling(10).mean())
)

#dd_feat_mov_avg.reset_index().compute()

# https://docs.dask.org/en/stable/generated/dask.dataframe.DataFrame.reset_index.html
# Note that unlike in pandas, the reset index for a Dask DataFrame will not be monotonically increasing from 0. 
# Instead, it will restart at 0 for each partition (e.g. index1 = [0, ..., 10], index2 = [0, ...]). 
# This is due to the inability to statically know the full length of the index.

C:\Users\The Winner\AppData\Local\Temp\ipykernel_2352\3233744760.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat_mov_avg = dd_feat.groupby('ticker', group_keys=False).apply(



Note aside:

After submission, in a branch named sandbox, I have picked a specific calculation, in this case the simple non-sense adding all the moving average column values from all tickers together, to find differences between methods. Things I learned:

- if we sort the original values (after groupby by ticker) by date, we obtain different result. Said that, I should have sorted the values by date inside and before each groupby calculation

- if we put together all the feature calculations in one single function and use map_partition, give different result, basically ensures that there will be 89 partitions (one per ticker) and that a ticker will not be split between different partitions, compromising the continuity of some calculations. Said that, I should have used map_partition

- if we use the one function and map_partition as described above, and we add the calculation of the feature moving_average inside such function, we have everything in Dask, no converting to Pandas, gives also different result. In here, the results sorted by dates and not sorted by dates are very similar. Because the issue with partitions, I assume this is the cleanest and best result.

- regarding calculation times, last option of all in Dask takes (in my computer, 4 threads) around 33 seconds (where Dask computes to give me only one value) vs around same 33 seconds of Dask+Pandas (where Dask computes all dataframe values to give it to Pandas). With this size of data there is no difference. 

- if the 3 different options are inside same Jupiter Notebook and click Run All, times get distorted and also results. Even if I use different variables name to avoid that, it reuses some calculations (I did not dig deep). When Restart, run one option, Restart, run second option, etc..., it gives always same results.

- the three different options (this assignement, using map_partions + pandas, using map_partitions Dask only), when we sort values by date grouped by ticker at the beginning, all three (3) give exact same result. When donig this sorting, we obtain three (3) different results

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.